In [1]:
%matplotlib inline
import time
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal
from scipy import io
import pickle

# Own code
sys.path.append("../")
from utils.data_utils import generate_dgp_tvp_var, generate_matrices, generate_contemp_matrices
from utils.tvp_models import TVPVARModel, tvp_ar_contemp, tvp_ar_non_contemp

# Suppress scientific notation in numpy
np.set_printoptions(suppress=True)

In [3]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 200
    p = 1
    train = 150
    
    prior_parameters = None
    
    if prior == 'lasso_alternative':
        prior_parameters = {'a0_lasso':1,'b0_lasso':1}
    elif prior == 'svss':
        prior_parameters = {'g0': 1, 'h0': 12, 'pi0': 0.5}
    elif prior == 'horseshoe':
        prior_parameters = {'a0': 1, 'b0': 1}
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{run}_het_py.csv', delimiter=",")
    
    # y, coefficients = generate_dgp_tvp_var(M, T, p, 1/2, 1/4, 4*1e-5, 1, 1/2, 1e-2)
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)

    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100, prior_parameters=prior_parameters)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run} & M: {M} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [ ]:
%%time
import time
import os
from multiprocessing import Pool, Array

# They are going to be some disgusting warnings for the first iterations of the model (has to do with initialization)
import warnings
warnings.filterwarnings("ignore")

np.random.seed(12345)
n_iterations = 200

iterations_set = np.arange(1,n_iterations+1,1)
M = (2,5)
prior_list = ["lasso_alternative", "svss", "horseshoe"]
args_tuples = [x for sub_list in [x for sub_list in [[[(n, m, prior) for n in iterations_set] for m in M] for prior in prior_list] for x in sub_list] for x in sub_list]

if __name__ == '__main__':
    
    pool = Pool()
    simulation_results = pool.starmap(simulation_run, args_tuples)

In [4]:
T = 200
train = 150
p = 1

simulation_parameters = [T,p,train, 1/2, 1/4, 4*1e-5, 1, 1/2, 1e-1*2, 1e-2, 1e-9, 12345, prior]

dump_to_disk = [simulation_results, simulation_parameters]

with open(f'../simulations/statistics_{T}_{p}_{n_iterations}_contemp_2.pkl', 'wb') as f:
        pickle.dump(dump_to_disk, f, pickle.HIGHEST_PROTOCOL)

In [17]:
a0 = np.linspace(1,10,10)
b0 = np.hstack(([1], np.linspace(2,10,5)))

msfe_list = []

for a_parameter in a0:
    
    for b_parameter in b0:

        T = 200
        p = 1
        train = 150
        prior = "lasso_alternative"
        M = 2
        run = 1

        y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{run}_het_py.csv', delimiter=",")
        coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{run}_het_py.csv', delimiter=",")

        y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)

        # Contemperanous values added 
        msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100, prior_parameters={'a0_lasso':a_parameter,'b0_lasso':b_parameter})
        msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)

        
        print(f"a0: {a_parameter} & b0: {b_parameter} -> MSFE: {round(msfe_contemp.mean(),6)}")
        msfe_list.append(msfe_contemp)

a0: 1.0 & b0: 1.0 -> MSFE: 2.438529
a0: 1.0 & b0: 2.0 -> MSFE: 2.433333
a0: 1.0 & b0: 4.0 -> MSFE: 2.438917
a0: 1.0 & b0: 6.0 -> MSFE: 2.444479
a0: 1.0 & b0: 8.0 -> MSFE: 2.449993
a0: 1.0 & b0: 10.0 -> MSFE: 2.455021
a0: 2.0 & b0: 1.0 -> MSFE: 2.441474
a0: 2.0 & b0: 2.0 -> MSFE: 2.435392
a0: 2.0 & b0: 4.0 -> MSFE: 2.438363
a0: 2.0 & b0: 6.0 -> MSFE: 2.44065
a0: 2.0 & b0: 8.0 -> MSFE: 2.445445
a0: 2.0 & b0: 10.0 -> MSFE: 2.449953
a0: 3.0 & b0: 1.0 -> MSFE: 2.443912
a0: 3.0 & b0: 2.0 -> MSFE: 2.43702
a0: 3.0 & b0: 4.0 -> MSFE: 2.435723
a0: 3.0 & b0: 6.0 -> MSFE: 2.440568
a0: 3.0 & b0: 8.0 -> MSFE: 2.442122
a0: 3.0 & b0: 10.0 -> MSFE: 2.445856
a0: 4.0 & b0: 1.0 -> MSFE: 2.445929
a0: 4.0 & b0: 2.0 -> MSFE: 2.438617
a0: 4.0 & b0: 4.0 -> MSFE: 2.434943
a0: 4.0 & b0: 6.0 -> MSFE: 2.4369
a0: 4.0 & b0: 8.0 -> MSFE: 2.439927
a0: 4.0 & b0: 10.0 -> MSFE: 2.44317
a0: 5.0 & b0: 1.0 -> MSFE: 2.447611
a0: 5.0 & b0: 2.0 -> MSFE: 2.440118
a0: 5.0 & b0: 4.0 -> MSFE: 2.434565
a0: 5.0 & b0: 6.0 -> MSFE: 2.

In [ ]:
T = 200
p = 1
train = 150
prior = "lasso_alternative"
M = 2
run = 1
a_parameter = 1
b_parameter = 1

y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{run}_het_py.csv', delimiter=",")
coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{run}_het_py.csv', delimiter=",")

y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)

# Contemperanous values added 
msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=True, iterations=100, prior_parameters={'a0_lasso':a_parameter,'b0_lasso':b_parameter})
msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)